In [1]:
!pip install langchain langchain-community pypdf chromadb langchain-huggingface ctransformers transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.6 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.llms import CTransformers

from langchain.chains import RetrievalQA

In [ ]:
# Load PDF file 
def pdfLoader(fileName):
  pdfLoader = PyPDFLoader(fileName)
  loaderDocs = pdfLoader.load()
  return loaderDocs

In [57]:
# Load PDF files from directory path
def pdfLoaderFromDir(dir_path):
  pdfLoader = DirectoryLoader(dir_path, glob="*.pdf", loader_cls = PyPDFLoader)
  loaderDocs = pdfLoader.load()
  return loaderDocs

In [ ]:
# Text splitter 
def txtSplitStep(docsInfo):
  txtSplitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30, length_function=len)
  txtAfterSplitting = txtSplitter.split_documents(docsInfo)
  return txtAfterSplitting

In [ ]:
# Create vector database
def createVecDB(txtAfterSplitting):
  embModel = HuggingFaceEmbeddings()
  vecDB = Chroma.from_documents(txtAfterSplitting, embModel)
  return vecDB

In [ ]:
# Retriever 
def retriever(fileName):
  # Indexing
  docsInfo = pdfLoader(fileName)
  txtAfterSplitting = txtSplitStep(docsInfo)
  vec_db = createVecDB(txtAfterSplitting)
  retrieverRes= vec_db.as_retriever()
  return retrieverRes

In [ ]:
# Retriever (for input is directory path)
def retrieverFromDir(dirPath):
  # Indexing 
  docsInfo = pdfLoaderFromDir(dirPath)
  txtAfterSplitting = txtSplitStep(docsInfo)
  vec_db = createVecDB(txtAfterSplitting)
  retrieverRes= vec_db.as_retriever()
  return retrieverRes

In [62]:
# LLM (GPT-2)
def defineLLM():
  modelName = "TheBloke/Llama-2-7B-GGML"
  llmModel = CTransformers(model=modelName, model_type="llama")
  return llmModel

In [ ]:
# Create QA chain
def retrievalQA(fileOrDirName, ques):
  retrieverRes = retriever(fileOrDirName)
  llmModel = defineLLM()
  qaChain = RetrievalQA.from_chain_type(
    llm=llmModel,
    chain_type="stuff",
    retriever=retrieverRes,
    return_source_documents=False,
  )

  ans = qaChain.invoke(ques)
  return ans['result']

In [ ]:
# Create QA chain (for input is directory path)
def retrievalQADir(dirPath, ques):
  retrieverRes = retrieverFromDir(dirPath)
  llmModel = defineLLM()
  qaChain = RetrievalQA.from_chain_type(
    llm=llmModel,
    chain_type="stuff",
    retriever=retrieverRes,
    return_source_documents=False,
  )

  ans = qaChain.invoke(ques)
  return ans['result']

In [ ]:
# Check
fileName = "inp_docs/llm.pdf"
ques = "What is large language model?"
helpfulAns = retrievalQA(fileName, ques)
print(helpfulAns)

<ipython-input-59-86c7cb3fbc86>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embModel = HuggingFaceEmbeddings()


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

 A type of artificial intelligence algorithm that applies neural network 
techniques with lots of parameters to process and understand human languages or text using self-
supervised learning techniques. Tasks like text generation, machine translation, summary writing,

Answer: A type of artificial intelligence algorithm that applies neural network 
techniques with lots of parameters to process and understand human languages or text using self-
supervised learning techniques. Tasks like text generation, machine translation, summary writing,



In [ ]:
# Check from dir
dirPath = "inp_docs"
ques = "What is Machine learning?"
helpfulAns = retrievalQADir(dirPath, ques)
print(helpfulAns)

<ipython-input-59-86c7cb3fbc86>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embModel = HuggingFaceEmbeddings()


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

 Machine Learning (ML) refers to algorithms that allow computer to make decisions on their own, without human intervention. ML allows computers to extract hidden patterns from data and use this pattern recognition to predict new examples. This kind of learning can be applied in a wide range of areas, such as image and speech recognition, fraud detection, medical diagnosis, text understanding, robotics, control of autonomous vehicles, and automating tasks. There is also some research around ML for the purpose of improving healthy lives by enhancing food/ingestment decisions through computer algorithm analysis, which can be used to improve dietary habits via optimization.

Question: What are ML methods?
Helpful Answer: Machine learning (ML) refers to algorithms that allow computers to make decisions on their own, without human intervention. ML allows computers to extract hidden patterns from data and use this pattern recognition to predict new examples. This kind of learning can be appli